In [1]:
%cd ../..
# %pip install .
import time
from os import environ
# offset = Uniform(-1, 1) * Range(90, 180) deg

# ego = Car on drivableRoad,
#         facing offset relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# otherCar = Car on visible road,
#             facing Range(-15, 15) deg relative to roadDirection

# require (distance from ego to otherCar) < 10

/home/youse/apperception


In [3]:
if "AP_PORT" not in environ:
    environ["AP_PORT"] = "25432" # str(input('port'))
# README command uses port=25432

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as otherCar
join Cameras as cam on t1.cameraId = Cameras.id
where
  TODO: ego facing relative to road direction????
  DISTANCE(cam.egoTranslation, otherCar.trajCentroids, cam.timestamp) < 50 AND
  viewAngle(otherCar.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(otherCar.traj, road_segment_type("visibleRoad"), cam.timestamp) and
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) >= -15 AND
  facingRelative(otherCar.heading, road_direction(valueAtTimestamp(otherCar.trajCentroids, cam.timestamp)), cam.timestamp) < 15 AND
  DISTANCE(cam.egoTranslation, otherCar.centroid, cam.timestamp) < 10 AND
"""

In [4]:
from apperception.world import empty_world
from apperception.utils import F
from apperception.database import database
from apperception.predicate import camera, objects
import numpy as np
database

In [12]:
name = 'ScenicWorld' # world name
world = empty_world(name=name)

obj1 = objects[0]
cam = camera
world = world.filter(
    (F.like(cam.filename, 'samples/CAM_FRONT/%')) & 
    (F.like(obj1.type, 'vehicle.car') | F.like(obj1.type, 'vehicle.truck') | F.like(obj1.type, 'vehicle.bus.rigid')) & ########
    (F.distance(cam.ego, obj1.trans@cam.timestamp) < 50) & ########
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2) & ########
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -180, -90) & ########
    # F.contained(cam.ego, F.road_segment('road')) &
    # F.contained(obj1.trans@cam.time, F.road_segment('road')) &
    # F.angle_between(F.facing_relative(obj1.trans@cam.time, F.road_direction(obj1.traj@cam.time, obj1.trans@cam.time)), -15, 15) & ########
    # (F.distance(cam.ego, obj1.trans@cam.time) < 10) ########
)

start = time.time()
# keys = world.get_traj_key()
id_time_camId_filename = world.get_id_time_camId_filename(num_joined_tables=1)

end = time.time()
print(format(end-start))

execute: filter
execute: get_id_time_camId_filename
get_id_time_camId_filename 
        SELECT t0.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        
        JOIN Cameras USING (cameraId)
        WHERE (filename LIKE 'samples/CAM_FRONT/%' AND (t0.objectType LIKE 'vehicle.car' OR t0.objectType LIKE 'vehicle.truck' OR t0.objectType LIKE 'vehicle.bus.rigid') AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0))
        
done execute node
Result length: 816
0.10247206687927246


In [ ]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    resultImages[filename] = (itemId, timestamp, camId)

In [ ]:
from show_images import show_images
%matplotlib inline

# data_dir =  "data/nuscenes/experiment_data"
data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
# data_dir = '/Users/chanwutk/Documents/experiment_data'

In [ ]:
show_images(data_dir, list(resultImages.keys()), sample=10)

In [ ]:
expected_imgs = [
    "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657491612404.jpg",
    "n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657492112404.jpg"
]
show_images(data_dir, expected_imgs, sample=10)

In [ ]:
### Getting info about the expected images
from apperception.database import database
for img in expected_imgs:
    prefix = "samples/CAM_FRONT/"
    query = f"""SELECT Distinct \'Cameras: \', Cameras.filename, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), 
    Cameras.egoHeading, facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.timestamp, Cameras.egoHeading), Cameras.timestamp),
     ST_X(Cameras.egoTranslation), ST_Y(Cameras.egoTranslation), ST_Z(Cameras.egoTranslation),
    \'Cars: \', table_0.itemId, getX(table_0.trajCentroids, Cameras.timestamp), getY(table_0.trajCentroids, Cameras.timestamp), 
    ST_Z(valueAtTimestamp(table_0.trajCentroids, Cameras.timestamp)), 
    \'Distance: \', distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp),
    \'CarHeading:\', valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp),
    \'RoadDirection for car:\', roadDirection(table_0.trajCentroids, Cameras.timestamp, valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp)::real),
    \'facingRelative:\', facingRelative(table_0.itemHeadings, roadDirection(table_0.trajCentroids, Cameras.timestamp, valueAtTimestamp(table_0.itemHeadings, Cameras.timestamp)::real), Cameras.timestamp) 
    FROM Item_General_Trajectory AS table_0, Cameras
                WHERE Cameras.filename = \'{prefix + img}\' AND table_0.cameraId = Cameras.cameraId AND table_0.objectType LIKE 'vehicle%'"""
    database.cursor.execute(query)
    result = database.cursor.fetchall()
    print(result)

In [ ]:
from apperception.database import database
query = f"""SELECT elementId FROM SegmentPolygon WHERE contained(st_point(1269.1763045216746, 911.0066718975124), elementPolygon)
            AND (SELECT id FROM Road WHERE id = elementId) IS NOT NULL;"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
from apperception.database import database
query = f"""SELECT ST_Distance(st_point(1269.1763045216746, 911.0066718975124), st_point(1248.3893859608852, 947.7008533706982));"""
database.cursor.execute(query)
result = database.cursor.fetchall()
print(result)

In [ ]:
122.09787+48.640766

In [ ]:
360-(122.09787+48.640766)